# 0. Import

In [2]:
import pickle               as pkl
import pandas               as pd
import numpy                as np
import plotly_express       as px
import matplotlib.pyplot    as plt

from sklearn import preprocessing   as pp
from sklearn import model_selection as ms

from boruta   import BorutaPy
from lightgbm import LGBMClassifier


## 0.1 Load Dataset

In [ ]:
df = pkl.load(open('../dataset/processed/4_data_preparation.pkl', 'rb'))

# 1.0 Boruta

In [ ]:
# Define Boruta - Essa peste demora 1 hora ou mais.
boruta = BorutaPy( rf, n_estimators = 'auto', verbose=2, random_state=42).fit( X_train_n, Y_train_n)

cols_selected = boruta.support_.tolist()

# best features
X_train_fs = X_train.drop( ['date', 'sales'], axis=1 )
cols_selected_boruta = X_train_fs.iloc[:, cols_selected].columns.to_list()

# not selected boruta
cols_not_selected_boruta = list( np.setdiff1d( X_train_fs.columns, cols_selected_boruta ) )

# 2.0 LGBM

In [ ]:
# Crie e treine o modelo LGBMClassifier
lgb_model = LGBMClassifier(n_jobs=-1, random_state=42)
lgb_model.fit(x_train, y_train)

# Obtenha as importâncias das características do modelo
importances = lgb_model.feature_importances_

# Ordene as características por importância decrescente
indices = np.argsort(importances)[::-1]

# Imprima o ranking das características
print('Feature ranking')
df = pd.DataFrame()

for i, j in zip(x_train.columns, importances):
    aux = pd.DataFrame({'feature': i, 'importance': j}, index=[0])
    df = pd.concat([df, aux], axis=0)

print(df.sort_values('importance', ascending=False))

# 3.0 Separação de treino e test com validação holdout

In [3]:
# Quais são as colunas que vou utilizar
features = ['idade', 'divida_atual', 'renda_anual']

# O que eu quero estudar / prever
label = ['saldo_atual']

In [5]:
random_state = 1

#Total (100%) -> Treino1(80%) e Teste(20%) (split = 0.2)
x_train, x_test, y_train, y_test = ms.train_test_split( features, label, test_size=0.2, random_state= random_state)

In [6]:
pkl.dump(x_test,open('../dataset/production/x_test','wb'))
pkl.dump(y_test,open('../dataset/production/y_test','wb'))

In [ ]:
#Treino2 (80%) -> Treino(60%) e Validação(20%) (split = 0.25)
x_train, x_val, y_train, y_val = ms.train_test_split( x_train, y_train, test_size=0.25, random_state= random_state)

In [8]:
pkl.dump(x_train,open('../dataset/production/x_train','wb'))
pkl.dump(y_train,open('../dataset/production/y_train','wb'))

pkl.dump(x_val,open('../dataset/production/x_val','wb'))
pkl.dump(y_val,open('../dataset/production/y_val','wb'))